In [178]:
import BaseBlocks
import Block_functions as bf
import pandas as pd
import numpy as np

In [179]:
train = pd.read_csv("../test_data/titanic_train.csv")
test = pd.read_csv("../test_data/titanic_test.csv")

In [180]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [181]:
# https://github.com/nyk510/vivid/blob/master/vivid/utils.py
from contextlib import contextmanager
from time import time

@contextmanager
def timer(logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None):
    if prefix: format_str = str(prefix) + format_str
    if suffix: format_str = format_str + str(suffix)
    start = time()
    yield
    d = time() - start
    out_str = format_str.format(d)
    if logger:
        logger.info(out_str)
    else:
        print(out_str)

In [187]:
def get_ce_features(input_df):
    _input_df = pd.concat([input_df], axis=1)
    cols = ["Sex"]
    encoder = BaseBlocks.CountEncodingBlock(cols=cols)
    output_df = encoder.fit(_input_df.astype(str))
    return output_df

In [194]:
bf.get_ce_features(train)

NameError: name 'BaseBlocks' is not defined

In [195]:
# propress
def to_features(train, test):
    input_df = pd.concat([train, test]).reset_index(drop=True)

    processes = [
        bf.get_ce_features,
    ]

    output_df = pd.DataFrame()
    for func in tqdm(processes):
        _df = func(input_df)
        assert len(_df) == len(input_df), func.__name__
        output_df = pd.concat([output_df, _df], axis=1)

    train_x = output_df.iloc[:len(train)] 
    test_x = output_df.iloc[len(train):].reset_index(drop=True)
    return train_x, test_x

In [196]:
train_x, test_x = to_features(train, test)

  0%|          | 0/1 [00:00<?, ?it/s]


NameError: name 'BaseBlocks' is not defined

In [150]:
def get_function(block, is_train):
    s = mapping = {
        True: 'fit',
        False: 'transform'
    }.get(is_train)
    return getattr(block, s)


def to_feature(input_df, 
               blocks,
               is_train=False):
    out_df = pd.DataFrame()

    for block in tqdm(blocks, total=len(blocks)):
        func = get_function(block, is_train)

        with timer(prefix='create ' + str(block) + ' '):
            _df = func(input_df)

        assert len(_df) == len(input_df), func.__name__
        out_df = pd.concat([out_df, _df], axis=1)

    return out_df

In [153]:
class CountEncodingBlock(BaseBlock):
    def __init__(self, column, whole_df: pd.DataFrame):
        self.column = column
        self.whole_df = whole_df

    def transform(self, input_df):
        output_df = pd.DataFrame()
        c = self.column

        vc = self.whole_df[c].value_counts()
        output_df[c] = input_df[c].map(vc)
        return output_df.add_prefix('CE_')
    
process_blocks = [
    *[BaseBlocks.CountEncodingBlock(c) for c in ['Sex']],
]

In [154]:
train_feat_df = to_feature(train, process_blocks, is_train=True)

  0%|          | 0/1 [00:00<?, ?it/s]


KeyError: 'Sex'

In [197]:
def get_mode(x):
    if len(x.dropna()) > 2:
        if len(x.dropna().mode()) > 1:
            result = np.full(len(x), x.dropna().mode()[0])
        else:
            result = np.full(len(x), x.dropna().mode())
    else:
        result = x

    return result

In [200]:
train["Name"].str.len()

0      23
1      51
2      22
3      44
4      24
       ..
886    21
887    28
888    40
889    21
890    19
Name: Name, Length: 891, dtype: int64